In [1]:
!pip install jiwer openai-whisper datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=6ad60d24134a587f6d3be0bc4da1618c3ec157

In [2]:
import whisper
from whisper.normalizers.basic import BasicTextNormalizer
from tqdm.notebook import tqdm
import pandas as pd
import jiwer

model = whisper.load_model("turbo")

100%|█████████████████████████████████████| 1.51G/1.51G [01:46<00:00, 15.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "facebook/covost2",
    "et_en",
    data_dir="/content/drive/My Drive/et",
    trust_remote_code=True,
)
print(raw_datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.4k [00:00<?, ?B/s]

covost2.py:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1782 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1576 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1571 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 1782
    })
    validation: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 1576
    })
    test: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 1571
    })
})


In [5]:
raw_datasets["train"].features

{'client_id': Value(dtype='string', id=None),
 'file': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'translation': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [6]:
raw_datasets["train"][:5]["file"]

['/content/drive/My Drive/et/clips/common_voice_et_18341764.mp3',
 '/content/drive/My Drive/et/clips/common_voice_et_18341765.mp3',
 '/content/drive/My Drive/et/clips/common_voice_et_18341766.mp3',
 '/content/drive/My Drive/et/clips/common_voice_et_18341767.mp3',
 '/content/drive/My Drive/et/clips/common_voice_et_18341768.mp3']

In [7]:
raw_datasets["train"][:5]["audio"]

[{'path': '/content/drive/My Drive/et/clips/common_voice_et_18341764.mp3',
  'array': array([ 1.38777878e-17, -2.77555756e-17, -6.93889390e-18, ...,
         -2.41477028e-06, -2.24877294e-06, -7.57916132e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/My Drive/et/clips/common_voice_et_18341765.mp3',
  'array': array([4.54747351e-12, 7.73070497e-12, 6.82121026e-12, ...,
         6.29576243e-06, 6.74936018e-06, 6.68263237e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/My Drive/et/clips/common_voice_et_18341766.mp3',
  'array': array([-2.72848411e-12, -7.27595761e-12,  0.00000000e+00, ...,
         -3.30899638e-06,  8.13507359e-06,  7.41065014e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/My Drive/et/clips/common_voice_et_18341767.mp3',
  'array': array([-3.27418093e-11, -3.63797881e-12, -4.36557457e-11, ...,
         -1.62289616e-05, -5.07203549e-06,  2.55643272e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/My Drive/et/clips/common_voi

In [8]:
raw_datasets["train"][:5]["sentence"]

['Ei, ei ja veelkord ei.',
 'Kõik kirjeldatud probleemid on Sirlile pähe toonud ka elust loobumise mõtteid.',
 'Lõpliku otsuse peaks FIS sportlaste suhtes tegema järgmisel nädalal.',
 'Puhkust on vaja taastumiseks, et tööd saaks teha selge peaga.',
 'Rakett või patarei on iga inimese enda valik.']

In [9]:
raw_datasets["train"][:5]["translation"]

['No, no, and again, no.',
 'All these problems had made Sirli consider ending her life.',
 'The FIS should make their final decision regarding the athletes next week.',
 'They needed the vacation to recover so that they could work with a clear head.',
 'A rocket or a battery is every person’s personal choice.']

In [10]:
data = raw_datasets["train"][:1000]

In [11]:
hypotheses = []
for sample in data["file"]:
    hypotheses.append((model.transcribe(sample, language="Estonian"))["text"])

In [12]:
data["hypothesis"] = hypotheses

In [13]:
data["hypothesis"][:5]

[' Ey, ey, ég vel hvort ey.',
 ' Kõik kirjeldatud probleimid on Sirlele pähe toonud ka elust loobumise mõtteid.',
 ' Lõpliku otsuse peaks viss sportlasa suhtes tegema järgmisel nädalal.',
 ' Puhkust on vaja taastamiseks, et tööd saaks teha selge peaga.',
 ' Rakett või battari on iga inimese enda valik.']

In [14]:
normalizer = BasicTextNormalizer()
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["sentence_clean"] = [normalizer(text) for text in data["sentence"]]
data["translation_clean"] = [normalizer(text) for text in data["translation"]]

In [15]:
wer = jiwer.wer(list(data["sentence_clean"]), list(data["hypothesis_clean"]))
print(f"WER: {wer * 100:.2f} %")

WER: 24.63 %


In [16]:
dataframe = pd.DataFrame(
    dict(
        transcription=data["hypothesis"],
        sentence=data["sentence"],
        translation=data["translation"],
        hypothesis_clean=data["hypothesis_clean"],
        sentence_clean=data["sentence_clean"],
        translation_clean=data["translation_clean"],
    )
)
pd.set_option("display.max_colwidth", None)
dataframe.head(1)

,transcription,sentence,translation,hypothesis_clean,sentence_clean,translation_clean
0,"Ey, ey, ég vel hvort ey.","Ei, ei ja veelkord ei.","No, no, and again, no.",ey ey ég vel hvort ey,ei ei ja veelkord ei,no no and again no


In [17]:
dataframe.to_csv("/content/drive/My Drive/dataset_asr.csv", index=False)